In [9]:
from datasets import load_dataset

dataset = "kmfoda/booksum"
maxlength = 8000
testdataset = load_dataset(dataset, split="test", cache_dir="../cache/")
avg_length = 0
max_id = 0
for i in range(len(testdataset)):
    avg_length += testdataset[i]["chapter_length"]
    if testdataset[i]["chapter_length"] > maxlength:
        maxlength = testdataset[i]["chapter_length"]
    if testdataset[i]["chapter_length"] < 5000:
        max_id = max_id + 1
print(maxlength)
print(avg_length/len(testdataset))
print(max_id)

89221.0
3953.744933612858
1172


In [17]:
import evaluate
from datasets import load_metric
rouge = load_metric("rouge")
predictions = ["hello goodbye lol lol", "ankh morpork"]
references = ["goodbye", "general kenobi duck duck"]
results = rouge.compute(predictions=predictions,
                        references=references,
                        )
print(results)

{'rouge1': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.125, recall=0.5, fmeasure=0.2), high=Score(precision=0.25, recall=1.0, fmeasure=0.4)), 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)), 'rougeL': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.125, recall=0.5, fmeasure=0.2), high=Score(precision=0.25, recall=1.0, fmeasure=0.4)), 'rougeLsum': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.125, recall=0.5, fmeasure=0.2), high=Score(precision=0.25, recall=1.0, fmeasure=0.4))}


In [ ]:
def create_dataset():
    val_dataset = load_dataset(dataset, split="validation", cache_dir="../cache/")
    test_dataset = load_dataset(dataset, split="test", cache_dir="../cache/")

    val_dataset = val_dataset.filter(lambda x: len(x["document"]) < maxlength)
    test_dataset = test_dataset.filter(lambda x: len(x["document"]) < maxlength)
    return val_dataset, test_dataset

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
model_name = "togethercomputer/LLaMA-2-7B-32K"

/usr/scratch/haokang/anaconda3/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_model():
    # 8bit quant
    tokenizer = AutoTokenizer.from_pretrained("togethercomputer/LLaMA-2-7B-32K",cache_dir="../cache/",padding=True,
        truncation=True,
        use_cache=True,
        max_length=6000)
    model = AutoModelForCausalLM.from_pretrained("togethercomputer/LLaMA-2-7B-32K", trust_remote_code=False, torch_dtype=torch.float16,cache_dir="../cache/")
    # model = model.half()
    return model, tokenizer

In [3]:
# val_dataset, test_dataset = create_dataset()
model, tokenizer = create_model()

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.30s/it]


In [6]:
prompt = ["K is not one of us at this time and we will not care about it","which one is our solder"]
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=20)
print(inputs)

tensor([[ 476,  338,  451,  697,  310,  502,  472,  445,  931,  322,  591,  674,
          451, 2562, 1048,  372,  607,  697,  338, 1749,  899,  672]])


In [ ]:
for i, data in enumerate(test_dataset):
    article = data["document"]
    print(len(article))
    prompt = (
                f"Please summarize the following article.\n\n{article}.\n\n"
            )
    inputs = tokenizer(prompt, return_tensors="pt")
    print(inputs["input_ids"].shape)
    output = model.generate(**inputs, max_length=128, temperature=0.7,do_sample=True,top_k=50,)
    print(output.shape)
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    # print(len(output_text))
    # print(output_text)
    break